# **Combining Candlestick Patterns for Stronger Trading Signals – Study Notes**

---

## **Overview**
Let's explore the possibility of improving the accuracy of trading signals by **combining multiple candlestick patterns** and **technical indicators**. The goal is to achieve higher predictive accuracy by creating combinations of patterns such as **bullish/bearish engulfing** and **rejection candles** (e.g., shooting star, hammer) along with technical indicators like **RSI**.

---

## **Key Concepts**

### **1. Candlestick Patterns Overview**
- **Bullish/Bearish Engulfing**: Indicates trend reversals. Accuracy expected: **60–80%** for daily charts.
- **Rejection Patterns** (Shooting Star, Hammer, Hanging Man):
  - Characterized by **long tails**.
  - Indicate strong price rejection against an ongoing trend.
  - Accuracy expected: **60–70%**.


---

## **2. Objective**
- Combine different **candlestick signals** to create a **stronger overall signal**.
- Examples:
  1. **Combination of 3 Candles**: Two rejection candles with long tails followed by an engulfing candle indicating a trend reversal.
  2. **Combination of 2 Candles**: One rejection candle with a long tail and an engulfing candle confirming the trend reversal.
  3. **Single Candle**: An engulfing pattern with a strong rejection tail.


<center>
  <img src="./images/candle-sticks.jpg" alt="Centered Image" width="400">
</center>
---

## **3. Data Loading and Preprocessing**
- **Dataset**: Euro vs USD daily chart (2003–2021).
- **Total Candles (Bars)**: **6631 bars**.
- **Data Cleaning**:
  - Remove rows with **zero volume** (weekends/holidays).
  - Reset the DataFrame index after cleaning.
- **Final Data**: **4733 rows** after cleaning.

---

## **4. Functions for Pattern Detection**

<center>
<img src= "./images/Shooting-Stars-and-Hanging-Man.png", alt text= "candlestick image" width ="400">
</center>

### **1. `isEngulfing(row)`**
- Tests if a candlestick is an **engulfing pattern**:
  - Returns `1`: Bearish Engulfing.
  - Returns `2`: Bullish Engulfing.
  - Returns `0`: No engulfing pattern.

### **2. `isEngulfingStrong(row)`**
- A stricter version of `isEngulfing()`:
  - Tests if the engulfing candle exceeds the **highest** or **lowest** value of the previous candle for a **stronger signal**.

### **3. `isStar(row)`**
- Detects **rejection patterns** (Shooting Star, Hammer, etc.).
  - Returns `1`: Rejection indicating a **downtrend**.
  - Returns `2`: Rejection indicating an **uptrend**.
  - Returns `0`: No rejection signal.
- **Key Parameters**:
  - **`ratio1`**: Defines how much longer the tail must be compared to the body (e.g., `1.5` or `2.0`).
  - **`ratio2`**: Defines the ratio between the upper and lower tails.

### **4. `direction(row)`**
- Determines the direction of the candlestick:
  - `1`: Downward (closing price < opening price).
  - `2`: Upward (closing price > opening price).
  - `0`: No clear direction (close = open).

---




---

## **5. Combining Signals**

A **signal column** is added to the DataFrame to store the combined results:
- Example conditions:
  - Bearish signal:
    ```python
    if isEngulfing(row) == 1 or isStar(row) == 1 and RSI < 30:
        signal = 1
    ```
  - Bullish signal:
    ```python
    if isEngulfing(row) == 2 or isStar(row) == 2 and RSI > 70:
        signal = 2
    ```
- **Signal Values**:
  - `1`: Downward trend.
  - `2`: Upward trend.
  - `0`: No clear trend.

### **Notes:**
- The  typical RSI logic:
  - **RSI < 30**: Strong downward trend (instead of predicting a reversal).
  - **RSI > 70**: Strong upward trend. 
  
  NB: they could also indicate reversals

---

In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot\notes\files\EURUSD_Candlestick_1_D_ASK_05.05.2003-30.06.2021.csv")

#Check if NA values are in data
df=df[df['volume']!=0] #Remove rows with 0 volume, no trading days
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.tail()

,Local time,open,high,low,close,volume
4729,24.06.2021 00:00:00.000 GMT+0300,1.19267,1.19565,1.19178,1.19322,85152.21000
4730,25.06.2021 00:00:00.000 GMT+0300,1.19322,1.19754,1.19264,1.19392,77837.64500
4731,28.06.2021 00:00:00.000 GMT+0300,1.19380,1.19447,1.19025,1.19260,85154.26000
4732,29.06.2021 00:00:00.000 GMT+0300,1.19297,1.19334,1.18779,1.18973,98898.57000
4733,30.06.2021 00:00:00.000 GMT+0300,1.18973,1.19092,1.18452,1.18589,4301.30191


In [ ]:
# check out key_levels.ipynb for details on key levels

# Support and Resitance FUNCTIONS
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    # for a support candle, it's low must be lower than the previous lows and preceding lows
    for i in range(l-n1+1, l+1):
        if(df1.low[i]>df1.low[i-1]): 
            return 0    # when there is a lower candle in the next candles( i+1), return 0
    for i in range(l+1,l+n2+1):
        if(df1.low[i]<df1.low[i-1]):
            return 0 
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    # for a resistance candle, it's high must be higher than the previous highs and preceding highs
    for i in range(l-n1+1, l+1):
        if(df1.high[i]<df1.high[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.high[i]>df1.high[i-1]):
            return 0
    return 1




In [4]:
length = len(df)
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length #

# bodydiff.head()
bodydiff[1] = abs(open[1]-close[1]) # the absolute difference between open and close
bodydiff 

[0,
 0.015700000000000047,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

In [ ]:
length = len(df)
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
open = list(df['open'])
bodydiff = [0] * length #

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row]) # the absolute difference between open and close
    if bodydiff[row]<0.000001:# 
        bodydiff[row]=0.000001 # if bodydiff is less than 0.000001, set it to 0.000001     

    bodydiffmin = 0.002
    # It first checks if the bodydiff values at indices row and row-1 are both greater than the bodydiffmin threshold.
    #  if the open price is less than the close price at index row-1, and if the open price is greater than the close price at index row.
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [ ]:
n1=2
n2=2
backCandles=45
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



In [ ]:
df['signal']=signal

df[df['signal']==1].count()



In [ ]:
SLTPRatio = 1.1 #TP/SL Ratio
def mytarget(barsupfront, df1):
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    signal = list(df1['signal'])
    trendcat = [0] * length
    amount = [0] * length
    
    SL=0
    TP=0
    for line in range(backCandles, length-barsupfront-n2):

        if signal[line]==1:
            SL = max(high[line-1:line+1])#!!!!! parameters
            TP = close[line]-SLTPRatio*(SL-close[line])
            for i in range(1,barsupfront+1):
                if(low[line+i]<=TP and high[line+i]>=SL):
                    trendcat[line]=3
                    break
                elif (low[line+i]<=TP):
                    trendcat[line]=1 #win trend 1 in signal 1
                    amount[line]=close[line]-low[line+i]
                    break
                elif (high[line+i]>=SL):
                    trendcat[line]=2 #loss trend 2 in signal 1
                    amount[line]=close[line]-high[line+i]
                    break

        if signal[line]==2:
            SL = min(low[line-1:line+1])#!!!!! parameters
            TP = close[line]+SLTPRatio*(close[line]-SL)
    
            for i in range(1,barsupfront+1):
                if(high[line+i]>=TP and low[line+i]<=SL):
                    trendcat[line]=3
                    break
                elif (high[line+i]>=TP):
                    trendcat[line]=2 #win trend 2 in signal 2
                    amount[line]=high[line+i]-close[line]
                    break
                elif (low[line+i]<=SL):
                    trendcat[line]=1 #loss trend 1 in signal 2
                    amount[line]=low[line+i]-close[line]
                    break
    #return trendcat
    return amount

df['Trend'] = mytarget(16, df)
df['Amount'] = mytarget(16, df)

## **6. Results and Signal Count**

- **Original Signal Count**:
  - Bearish Signals: **17**
  - Bullish Signals: **27**
  - Total: **44 signals** over **18 years** (~2–3 signals per year).

- **After Removing RSI Condition**:
  - Bearish Signals: **315**
  - Bullish Signals: **298**
  - Total: **~600 signals** over **18 years** (~33 signals per year).

---

## **7. Testing Prediction Accuracy**

The **myTarget()** function checks if the price moves up or down by a given number of pips (`pip_limit`) within a certain number of future days (`n`):
- Example:
  - `pip_limit`: **30 pips**.
  - `n`: **10 days**.

**Return Values:**
- `1`: Price dropped by `pip_limit`.
- `2`: Price increased by `pip_limit`.
- `3`: Price moved both ways.
- `0`: No significant movement.

---

## **8. Performance Results**

### **Initial Testing (30 Pips, 10 Days)**
- **Upward Trend**: **67% accuracy**.
- **Downward Trend**: **66% accuracy**.

### **With RSI Conditions**
- **Upward Trend**: **85% accuracy**.
- **Downward Trend**: **82% accuracy**.
- **Total Signals**: **2–3 signals per year**.

### **Shorter Profit Target (20 Pips)**
- **Accuracy**: **92%**.
  - **Note**: Shorter profit targets increase the likelihood of reaching the target.

---

## **9. False Positives Analysis**
- False positives occur when the signal direction does not match the actual price movement:
  - **Example**:
    - Bar 2969: Correct bearish engulfing pattern with a downtrend.
    - Bar 2979: False bearish engulfing pattern (followed by an upward trend).

---

## **10. Considerations for Strategy Building**

- **Number of Signals**:
  - Combining too many conditions (e.g., engulfing + rejection + RSI) can reduce the number of signals.
  - Example: Daily charts may only yield **7 signals** over **18 years**.
  - **Smaller timeframes (e.g., 4-hour charts)** can provide more frequent signals.

- **Risk Management**:
  - Proper **stop-loss** and **take-profit** placement is essential.
  - Avoid relying on very few signals (e.g., 1–2 signals per year).

---

## **Key Takeaways**

- Combining candlestick patterns with indicators can significantly improve accuracy but may reduce the number of signals.
- **RSI levels** can be interpreted differently depending on timeframe and strategy.
- Be cautious of **overfitting conditions**—more conditions can result in fewer signals.
- Accuracy increases with shorter profit targets, but realistic **risk-reward ratios** must be maintained.